# JSON-LD Conversions

This notebook demonstrates how to [convert](https://nexus-forge.readthedocs.io/en/latest/interaction.html#converting) Resources to [JSON-LD](https://json-ld.org) and vice-versa. JSON-LD is a semantic-preserving JSON format allowing to provide identifiers and meaning to JSON keys and values by mean of an added '@context' object. Read the excellent [JSON-LD documentation](https://json-ld.org/learn.html) to learn more about this format.

A JSON-LD context can be assigned to a [Resource](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource) in three ways by order of priority:
1. directly set in the resource using the property `context`: e.g.`jane_resource.context="https://schema.org/docs/jsonldcontext.json"`
2. set in the `Model` section of the forge configuration file: e.g.


'''

Model:

     name: RdfModel
  
     origin: ...
  
     source: ...
  
     context:
  
         iri: "https://schema.org/docs/jsonldcontext.json"
    
''' 


3. set by the configured store. A store (such as BlueBrainNexus) can support JSON-LD and add a default JSON-LD context whenever one is not provided.

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

# Imports

In [3]:
import json

In [4]:
from kgforge.core import Resource

In [5]:
def pp(x): 
    print(json.dumps(x, indent=4))

## Context

In [6]:
context = {
    "ex": "http://example.org/",
    "Person": "ex:Person",
    "Organization": "ex:Organization",
    "employer": "ex:employer",
    "name": "ex:name",
    "award":"ex:award",
    "email":"ex:email"
}

## Resource to JSON-LD

### context from the user

#### locally defined context

In [7]:
jane = Resource(context=context, type="Person", name="Jane Doe", award=["Nobel"], email=["jane.doe1@example.org", "jane.doe2@example.org"])

In [8]:
forge.as_json(jane)

{'type': 'Person',
 'award': ['Nobel'],
 'email': ['jane.doe1@example.org', 'jane.doe2@example.org'],
 'name': 'Jane Doe'}

In [9]:
pp(forge.as_jsonld(jane)) # default JSON-LD form=="compacted"

{
    "@context": {
        "ex": "http://example.org/",
        "Person": "ex:Person",
        "Organization": "ex:Organization",
        "employer": "ex:employer",
        "name": "ex:name",
        "award": "ex:award",
        "email": "ex:email"
    },
    "@type": "Person",
    "name": "Jane Doe",
    "award": [
        "Nobel"
    ],
    "email": [
        "jane.doe1@example.org",
        "jane.doe2@example.org"
    ]
}


In [10]:
pp(forge.as_jsonld(jane, form="expanded"))

{
    "@type": [
        "http://example.org/Person"
    ],
    "http://example.org/award": [
        {
            "@value": "Nobel"
        }
    ],
    "http://example.org/email": [
        {
            "@value": "jane.doe1@example.org"
        },
        {
            "@value": "jane.doe2@example.org"
        }
    ],
    "http://example.org/name": [
        {
            "@value": "Jane Doe"
        }
    ]
}


In [11]:
pp(forge.as_jsonld(jane, form="compacted"))

{
    "@context": {
        "ex": "http://example.org/",
        "Person": "ex:Person",
        "Organization": "ex:Organization",
        "employer": "ex:employer",
        "name": "ex:name",
        "award": "ex:award",
        "email": "ex:email"
    },
    "@type": "Person",
    "name": "Jane Doe",
    "award": [
        "Nobel"
    ],
    "email": [
        "jane.doe1@example.org",
        "jane.doe2@example.org"
    ]
}


#### remote context from the web

In [14]:
jane = Resource(context="https://schema.org/docs/jsonldcontext.json", type="Person", name="Jane Doe")

In [15]:
print(jane)

{
    context: https://schema.org/docs/jsonldcontext.json
    type: Person
    name: Jane Doe
}


In [16]:
pp(forge.as_jsonld(jane, form="expanded"))

{
    "@type": [
        "http://schema.org/Person"
    ],
    "http://schema.org/name": [
        {
            "@value": "Jane Doe"
        }
    ]
}


### context from the model

The configured model provides a default context that will be used to create resources that do not have context provided.

In [17]:
jane = Resource(type="Person", name="Jane Doe")

In [18]:
print(jane)

{
    type: Person
    name: Jane Doe
}


In [19]:
pp(forge.as_jsonld(jane))

{
    "@context": "https://bbp.neuroshapes.org",
    "@type": "Person",
    "name": "Jane Doe"
}


In [20]:
pp(forge.as_jsonld(jane, form="expanded"))

{
    "@type": [
        "http://schema.org/Person"
    ],
    "http://schema.org/name": [
        {
            "@value": "Jane Doe"
        }
    ]
}


### context from the Store

It is possible to use a context that is available in the configured RdfModel Store

## JSON-LD to Resource

In [21]:
john = Resource(context=context, type="Person", name="John Smith", award=["A.M. TURING AWARD"], 
                email=["john.smith1@example.org", "john.smith2@example.org"])

In [22]:
data = {
    "@context": context,
    "@type": "Person",
    "name": "John Smith",
    "award":["A.M. TURING AWARD"],
    "email":["john.smith1@example.org", "john.smith2@example.org"]
}

In [23]:
resource = forge.from_jsonld(data)

In [24]:
resource == john

True